# Sudoku Solver - Colab Example

Приклад використання всіх моделей (CNN, GNN, RNN) в Google Colab

## 1. Встановлення залежностей

In [ ]:
# Встановлюємо PyTorch та інші залежності
!pip install torch torchvision torch-geometric fastapi uvicorn pandas numpy scikit-learn tqdm python-multipart pydantic

## 2. Завантаження файлів проєкту

Завантажте файли проєкту в Colab (через Google Drive або завантаження)

In [ ]:
# Варіант 1: З Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Змініть шлях до вашої папки
# import sys
# sys.path.append('/content/drive/MyDrive/backend')

In [ ]:
# Варіант 2: Завантаження файлів безпосередньо
# Завантажте файли: models/, dataset.py, train.py через файловий менеджер Colab

## 3. Імпорт моделей

In [ ]:
import torch
import numpy as np
from models import CNNBaseline, CNNAdvanced, GNNModel, SudokuRNN

print("✓ Моделі успішно імпортовано")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 4. Створення тестового пазлу

In [ ]:
def example_puzzle():
    """Приклад судоку пазлу"""
    puzzle_str = "530070000600195000098000060800060003400803001700020006060000280000419005000080079"
    puzzle = np.array([int(c) for c in puzzle_str]).reshape(9, 9)
    return puzzle

puzzle = example_puzzle()
print("Тестовий пазл:")
print(puzzle)

## 5. Тестування моделей

In [ ]:
def solve_with_model(model, puzzle, model_name=None):
    """Вирішення пазлу з використанням моделі"""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # RNN потребує flattened input
    if model_name == 'rnn' or isinstance(model, SudokuRNN):
        puzzle_tensor = torch.from_numpy(puzzle.flatten()).unsqueeze(0).long().to(device)
    else:
        puzzle_tensor = torch.from_numpy(puzzle).unsqueeze(0).long().to(device)
    
    model = model.to(device)
    model.eval()
    
    with torch.no_grad():
        output = model(puzzle_tensor)
        probabilities = torch.softmax(output, dim=-1)
        confidence, predictions = torch.max(probabilities, dim=-1)
        predictions = predictions + 1  # 0-8 -> 1-9
    
    solution = predictions[0].cpu().numpy()
    avg_confidence = confidence.mean().item()
    
    return solution, avg_confidence

# Створюємо моделі
models = {
    'Baseline CNN': (CNNBaseline(hidden_channels=64), None),
    'Advanced CNN': (CNNAdvanced(hidden_channels=128, num_residual_blocks=20), None),
    'GNN': (GNNModel(hidden_channels=128, num_layers=6), None),
    'RNN': (SudokuRNN(embedding_dim=64, hidden_size=128, num_layers=2), 'rnn'),
}

# Тестуємо кожну модель
for model_name, (model, model_type) in models.items():
    print(f"\n{'='*50}")
    print(f"{model_name}")
    print(f"Параметри: {sum(p.numel() for p in model.parameters()):,}")
    
    solution, confidence = solve_with_model(model, puzzle, model_type)
    print(f"Впевненість: {confidence:.2%}")
    print(f"Рішення (перші 3 рядки):\n{solution[:3]}")

## 6. Тренування моделі RNN

In [ ]:
# Приклад тренування RNN моделі
# !python train.py --model rnn --epochs 20 --batch-size 64 --device cuda

# Або в Colab можна запустити так:
# import subprocess
# subprocess.run([
#     'python', 'train.py',
#     '--model', 'rnn',
#     '--epochs', '20',
#     '--batch-size', '64',
#     '--device', 'cuda'
# ])

## 7. Завантаження навченої моделі

In [ ]:
# Приклад завантаження навченої RNN моделі
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SudokuRNN(embedding_dim=64, hidden_size=128, num_layers=2)
model = model.to(device)

# Завантажити ваги (якщо вони є)
# checkpoint_path = 'weights/rnn_best.pth'
# if os.path.exists(checkpoint_path):
#     checkpoint = torch.load(checkpoint_path, map_location=device)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     print(f"✓ Модель завантажено з епохи {checkpoint.get('epoch', 'unknown')}")
# else:
#     print("⚠ Ваги не знайдено, використовується ненавчена модель")

model.eval()
print("✓ Модель готова до використання")

## Важливі примітки для Colab:

1. **RNN модель працює так само, як CNN та GNN** - всі моделі використовують стандартні PyTorch компоненти
2. **RNN не потребує torch-geometric** (на відміну від GNN), тому вона навіть простіша для встановлення
3. **Єдина відмінність**: RNN потребує flattened input `(batch, 81)` замість `(batch, 9, 9)`
4. **Всі моделі повертають однаковий формат виводу**: `(batch, 9, 9, 9)`
5. **Тренування**: Використовуйте той самий `train.py` з аргументом `--model rnn`